# Notebook to fit amplitudes to Lensing Signals

This was the program used in the paper

Runs the ampltidues for wtot, Wsys, and no weight.

Makes the files called "amplitudes"

e.g.  :

LOWZ_0.15_0.31_cweight_wtot.txt => these are the weights being printed out

LOWZ_0.15_0.31_amplitude_errs_wtot.txt => errors on amplitudes

LOWZ_0.15_0.31_amplitudes_wtot.txt => these are the amplitudes

In [1]:
import numpy as np
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
# The HOD model file
# see interpolate.pro
modelfile = "hod_model_saito2016_mdpl2.txt"       # model for weighting
temp = np.genfromtxt(modelfile)
model = temp[:,1]
rbins = temp[:,0]

In [3]:
# *** Change here depending on weight type ***
# Run once for each option
weight_type = "_wtot"
#weight_type = "_now"

In [4]:
smallr ,= np.where(rbins<1.) # 1 Mpc split
larger ,= np.where(rbins>1.)
allr ,= np.where(rbins<50)

binranges = []
binranges.append(smallr)
binranges.append(larger)
binranges.append(allr)

In [5]:
# Function that computes amplitudes

def amplitudes(measurements, covariances, model, binranges, experiments, file1, file2, file3, file4):
    # measurements is a dictionary of Nd vectors, 
    # covariances is a dictionary of NxN matrixes, 
    # model is a Nd vector, 
    # binranges is a list of integer vectors, 
    # experiments is a list of names 
    
    for binrange in binranges:
        
        print("------ Bin range -----",binrange)
        print("bin range ",binrange)
        
        # pick relevant entries from measurements, covariances, model
        meas = {}
        cov = {}
        invcov = {}
        weight = {} # optimal weight for that survey
        denom  = {} # the denominator for that survey that brings A=1 if meas=mod
        mod = model[binrange]
        #print("model",mod)
        
        # sum of covariances of that set of bins
        sumcov = np.zeros((len(binrange),len(binrange)))
        
        #print("\n----- Calculating weights -------\n")
            
        for experiment in experiments:
            
            #print(experiment)
            meas[experiment] = measurements[experiment][binrange]
            cov[experiment]  = covariances[experiment][binrange][:,binrange]
            invcov[experiment] = np.linalg.inv(cov[experiment])
            weight[experiment] = np.dot(invcov[experiment],mod)
            print("experiment",experiment)
            #type(weight[experiment])
            print("weight",weight[experiment])
            
            denom[experiment] = np.dot(weight[experiment],mod)
            
            # sum all covs for joint weighting
            sumcov += cov[experiment]
            
        # optimal weight for comparison is inverse of sum of covariances times model
        cweight = np.dot(np.linalg.inv(sumcov),mod)
        cdenom   = np.dot(cweight,mod) # normalizing the amplitude to 1 if measurement = model
        
        print("cweight",cweight)
        print("cdenom",cdenom)      
        
        #print("\n----- Calculating amplitudes -------\n")
        
        for experiment in experiments:
            a    = np.dot(meas[experiment],cweight)/cdenom
            siga = np.sqrt(np.dot(np.dot(cweight,cov[experiment]),cweight)/cdenom**2)
            #print(experiment,a,"+-",siga)
            
            # Write out amplitudes
            file1.write('%0.4f' % a)
            file1.write(' ')

            # Write out errors
            file2.write('%0.4f' % siga)
            file2.write(' ')
        
            #print("a and siga", a, siga)
   
            #print("with its own optimal weighting the experiment",experiment,"would have given us",a,"+-",siga)
            a    = np.dot(meas[experiment],weight[experiment])/denom[experiment]
            siga = np.sqrt(np.dot(np.dot(weight[experiment],cov[experiment]),weight[experiment])/denom[experiment]**2)
   
        # also print out the weights
        # to check if they have imporatnt variations with R
        #print('cweight',cweight)
        
        for item in cweight:
            file3.write("%0.4f" % item)
            file3.write(' ')
            
        file4.write('%0.4f' % cdenom)
                
        file1.write('\n')
        file2.write('\n')
        file3.write('\n')
        file4.write('\n')
            
    file1.close()  
    file2.close() 
    file3.close()
    file4.close()

In [6]:
# This one reuses an exsiting weight function from a previous calculation

def amplitudes_existingwht(measurements, covariances, model, binranges, experiments, file1, file2,cwhtfile,denomfile):
    # measurements is a dictionary of Nd vectors, 
    # covariances is a dictionary of NxN matrixes, 
    # model is a Nd vector, 
    # binranges is a list of integer vectors, 
    # experiments is a list of names 
    
    # Reuse cweight and cdenom from fiducial runs
    # Read in the files here
    
    # This file has lines of different lengths
    # Read line by line instead
    fp = open(cwhtfile)    
    cdenomarray = np.genfromtxt(denomfile)
        
    i=0
    
    for binrange in binranges:
        
        # pick relevant entries from measurements, covariances, model
        meas = {}
        cov = {}
        invcov = {}
        weight = {} # optimal weight for that survey
        denom  = {} # the denominator for that survey that brings A=1 if meas=mod
        mod = model[binrange]

        # cweight depends on radial range
        # Read in line by line here

        line = fp.readline()
        x=line.split()
        n=len(x)

        cweight=np.zeros(n)
        for j in range(0,n):
            cweight[j]=float(x[j])
        
        print("cweights",cweight)
        cdenom = cdenomarray[i]
        i=i+1
        
        for experiment in experiments:
            
            meas[experiment] = measurements[experiment][binrange]
            cov[experiment]  = covariances[experiment][binrange][:,binrange]
            invcov[experiment] = np.linalg.inv(cov[experiment])
        
        for experiment in experiments:
        
            a    = np.dot(meas[experiment],cweight)/cdenom
            siga = np.sqrt(np.dot(np.dot(cweight,cov[experiment]),cweight)/cdenom**2)
            #print(experiment,a,"+-",siga)
            
            # Write out amplitudes
            file1.write('%0.4f' % a)
            file1.write(' ')

            # Write out errors
            file2.write('%0.4f' % siga)
            file2.write(' ')
                
        file1.write('\n')
        file2.write('\n')
         
    file1.close()  
    file2.close() 

# Order of Experiments

In [7]:
# define filenames / filename parts for various runs
experiments = ["DES","CFHT","HSC","CS82","KiDS","SDSS"] 

# LOWZ (includes SDSS)

In [8]:
sample = ["LOWZ",0.15,0.31]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

DES_LOWZ_0.15_0.31_wtot_cov.dat
DES_LOWZ_0.15_0.31_wtot.dat
CFHT_LOWZ_0.15_0.31_wtot_cov.dat
CFHT_LOWZ_0.15_0.31_wtot.dat
HSC_LOWZ_0.15_0.31_wtot_cov.dat
HSC_LOWZ_0.15_0.31_wtot.dat
CS82_LOWZ_0.15_0.31_wtot_cov.dat
CS82_LOWZ_0.15_0.31_wtot.dat
KiDS_LOWZ_0.15_0.31_wtot_cov.dat
KiDS_LOWZ_0.15_0.31_wtot.dat
SDSS_LOWZ_0.15_0.31_wtot_cov.dat
SDSS_LOWZ_0.15_0.31_wtot.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.99400054 0.36467989 3.46830867 3.64128605 9.09533198]
experiment CFHT
weight [ 0.76715423  1.48268536  3.17613216  5.99864392 11.32316102]
experiment HSC
weight [0.66950686 2.99770995 5.16668666 8.35952051 5.68528238]
experiment CS82
weight [0.86421482 0.65405735 6.69775744 8.66018763 1.20459091]
experiment KiDS
weight [0.32534035 0.99847058 0.758751   3.37180298 6.22304725]
experiment SDSS
weight [ 2.15881831  2.62892572  4.52575822 11.80781981 19.72121839]
cweight [0.11955903 0.2

In [9]:
sample = ["LOWZ",0.31,0.43]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS, SDSS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

DES_LOWZ_0.31_0.43_wtot_cov.dat
DES_LOWZ_0.31_0.43_wtot.dat
CFHT_LOWZ_0.31_0.43_wtot_cov.dat
CFHT_LOWZ_0.31_0.43_wtot.dat
HSC_LOWZ_0.31_0.43_wtot_cov.dat
HSC_LOWZ_0.31_0.43_wtot.dat
CS82_LOWZ_0.31_0.43_wtot_cov.dat
CS82_LOWZ_0.31_0.43_wtot.dat
KiDS_LOWZ_0.31_0.43_wtot_cov.dat
KiDS_LOWZ_0.31_0.43_wtot.dat
SDSS_LOWZ_0.31_0.43_wtot_cov.dat
SDSS_LOWZ_0.31_0.43_wtot.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.21957588 0.74084511 0.62402247 1.70973407 2.33880449]
experiment CFHT
weight [0.40524579 1.17265804 2.09102155 4.12037627 7.01997724]
experiment HSC
weight [0.77876693 1.27571096 4.0751884  2.83290825 6.0610666 ]
experiment CS82
weight [0.44728786 0.68148261 0.94117522 2.11094804 3.53717593]
experiment KiDS
weight [0.22050015 0.39648198 0.77658533 1.68935911 4.10975011]
experiment SDSS
weight [ 0.37460046  0.58541984  3.10977431  3.57981681 13.42253484]
cweight [0.05780003 0.116168

# CMASS - no SDSS here

In [10]:
experiments = ["DES","CFHT","HSC","CS82","KiDS"] 

In [11]:
sample = ["CMASS",0.43,0.54]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")



 ---- Files names ---- 

DES_CMASS_0.43_0.54_wtot_cov.dat
DES_CMASS_0.43_0.54_wtot.dat
CFHT_CMASS_0.43_0.54_wtot_cov.dat
CFHT_CMASS_0.43_0.54_wtot.dat
HSC_CMASS_0.43_0.54_wtot_cov.dat
HSC_CMASS_0.43_0.54_wtot.dat
CS82_CMASS_0.43_0.54_wtot_cov.dat
CS82_CMASS_0.43_0.54_wtot.dat
KiDS_CMASS_0.43_0.54_wtot_cov.dat
KiDS_CMASS_0.43_0.54_wtot.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.13222712 0.45796285 0.58679766 2.30114639 4.20802803]
experiment CFHT
weight [0.47198763 1.07800105 2.18427521 4.67421065 8.52434992]
experiment HSC
weight [0.95068414 2.15803345 3.17848503 5.03696794 9.21805717]
experiment CS82
weight [0.21721251 0.06536605 0.89524684 1.78312861 2.64895058]
experiment KiDS
weight [0.31958559 0.63416094 0.72061581 1.65079874 3.17334851]
cweight [0.05589746 0.09131912 0.19497721 0.49105027 0.82657935]
cdenom 59.90091409658556
------ Bin range ----- [5 6 7 8 9]
bin range  [5 6 7 8 9]
experiment DES
we

In [12]:
sample = ["CMASS",0.54,0.70]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for experiment in experiments:
    fcov = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+"_cov.dat"
    print(fcov)
    covariances[experiment] = np.genfromtxt(fcov)

    # Make sure read in DS here, there are many columns in the DS results files
    fdat = experiment+"_"+sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+str(weight_type)+".dat"
    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiment]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes"+str(weight_type)+".txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_amplitudes_errs"+str(weight_type)+".txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: DES, CFHT, HSC, CS82, KiDS \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Weights
file3 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt",'w')
file4 = open(sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt",'w')

amplitudes(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

DES_CMASS_0.54_0.7_wtot_cov.dat
DES_CMASS_0.54_0.7_wtot.dat
CFHT_CMASS_0.54_0.7_wtot_cov.dat
CFHT_CMASS_0.54_0.7_wtot.dat
HSC_CMASS_0.54_0.7_wtot_cov.dat
HSC_CMASS_0.54_0.7_wtot.dat
CS82_CMASS_0.54_0.7_wtot_cov.dat
CS82_CMASS_0.54_0.7_wtot.dat
KiDS_CMASS_0.54_0.7_wtot_cov.dat
KiDS_CMASS_0.54_0.7_wtot.dat

 ---- Now run the amplitudes ---- 

------ Bin range ----- [0 1 2 3 4]
bin range  [0 1 2 3 4]
experiment DES
weight [0.1559451  0.15268378 0.46236802 0.87150111 1.28218432]
experiment CFHT
weight [0.31336683 0.62098485 1.39950333 3.36008126 5.09046963]
experiment HSC
weight [ 1.57702984  4.87375432  5.74251435 23.27755314 25.58213684]
experiment CS82
weight [0.09593437 0.04501543 0.42954392 0.82233804 1.18983953]
experiment KiDS
weight [0.17121187 0.24339538 0.52812201 0.80377211 1.74262578]
cweight [0.03693358 0.03886382 0.13243269 0.25235943 0.40449449]
cdenom 32.51634413769284
------ Bin range ----- [5 6 7 8 9]
bin range  [5 6 7 8 9]
experiment DES
weight 

# Post blinding tests

In [13]:
# Nstar internal tests from Song using HSC
#HSC_CMASS_0.54_0.70_wtot_nstar_low.dat
#HSC_CMASS_0.43_0.54_wtot_nstar_low_cov.dat	
#HSC_CMASS_0.54_0.70_wtot_nstar_low_cov.dat
#HSC_CMASS_0.43_0.54_wtot_nstar_upp.dat		
#HSC_CMASS_0.54_0.70_wtot_nstar_upp.dat
#HSC_CMASS_0.43_0.54_wtot_nstar_upp_cov.dat	
#HSC_CMASS_0.54_0.70_wtot_nstar_upp_cov.dat

experiments = ["HSClow","HSCupp","HSCall"] 
sample = ["CMASS",0.54,0.70]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
for i in range(0,3):
  
    if(i==0):
        fcov = "HSC_CMASS_0.54_0.70_wtot_nstar_low_cov.dat"
        fdat = "HSC_CMASS_0.54_0.70_wtot_nstar_low.dat"
    
    if(i==1):
        fcov = "HSC_CMASS_0.54_0.70_wtot_nstar_upp_cov.dat"
        fdat = "HSC_CMASS_0.54_0.70_wtot_nstar_upp.dat"
        
    if(i==2):
        fcov = "HSC_CMASS_0.54_0.7_wtot_cov.dat"
        fdat = "HSC_CMASS_0.54_0.7_wtot.dat"
        
    print("fcov",fcov)
    covariances[experiments[i]] = np.genfromtxt(fcov)

    print(fdat)
    temp = np.genfromtxt(fdat)
    
    measurements[experiments[i]]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open("HSC_CMASS_0.54_0.70_nstartests_amplitudes.txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: low nstar, high nstar, all \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open("HSC_CMASS_0.54_0.70_nstartests_amplitudes_errs.txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: low nstar, high nstar \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Resuse existing weight here!!
file3 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt"
file4 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt"

# This uses existing weights computed previoulsy
amplitudes_existingwht(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

fcov HSC_CMASS_0.54_0.70_wtot_nstar_low_cov.dat
HSC_CMASS_0.54_0.70_wtot_nstar_low.dat
fcov HSC_CMASS_0.54_0.70_wtot_nstar_upp_cov.dat
HSC_CMASS_0.54_0.70_wtot_nstar_upp.dat
fcov HSC_CMASS_0.54_0.7_wtot_cov.dat
HSC_CMASS_0.54_0.7_wtot.dat

 ---- Now run the amplitudes ---- 

cweights [0.0369 0.0389 0.1324 0.2524 0.4045]
cweights [0.7572 0.9776 1.2718 1.3551 2.9717]
cweights [0.0302 0.0314 0.1419 0.2291 0.386  0.762  0.4685 1.216  1.0865 3.0738]
done !


In [14]:
# post-blinding test
# Adding in SDSS for C1 

In [15]:
# SDSS_CMASS_0.43_0.54_wtot.dat
# SDSS_CMASS_0.43_0.54_wtot_cov.dat

experiments = ["SDSS"] 
sample = ["CMASS",0.54,0.70]

measurements = {}
covariances = {}
           
print('\n ---- Files names ---- \n')
    
i=0

fcov = "SDSS_CMASS_0.43_0.54_wtot_cov.dat"
fdat = "SDSS_CMASS_0.43_0.54_wtot.dat"
    
print("fcov",fcov)
covariances[experiments[i]] = np.genfromtxt(fcov)

print(fdat)
temp = np.genfromtxt(fdat)
    
measurements[experiments[i]]= temp[:,1] # second column in the data file
    
print('\n ---- Now run the amplitudes ---- \n')

# Now run the Amplitudes
# Where the amplitudes are recorded
file1 = open("SDSS_CMASS_0.43_0.54_amplitudes.txt",'w')
file1.write('# Amplitudes \n')
file1.write('# cols: low z, high z, all \n')
file1.write('# row 1: small R, row2: large R, row3: all R \n')

# Amplitude errors
file2 = open("SDSS_CMASS_0.43_0.54_amplitudes_errs.txt",'w')
file2.write('# Errors on amplitudes \n')
file2.write('# cols: low z, high z \n')
file2.write('# row 1: small R, row2: large R, row3: all R \n')

# Resuse existing weight here!!
file3 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cweight"+str(weight_type)+".txt"
file4 = sample[0]+"_"+str(sample[1])+"_"+str(sample[2])+"_cdenom"+str(weight_type)+".txt"

# This uses existing weights computed previoulsy
amplitudes_existingwht(measurements,covariances,model,binranges,experiments,file1,file2,file3,file4)  
print("done !")


 ---- Files names ---- 

fcov SDSS_CMASS_0.43_0.54_wtot_cov.dat
SDSS_CMASS_0.43_0.54_wtot.dat

 ---- Now run the amplitudes ---- 

cweights [0.0369 0.0389 0.1324 0.2524 0.4045]
cweights [0.7572 0.9776 1.2718 1.3551 2.9717]
cweights [0.0302 0.0314 0.1419 0.2291 0.386  0.762  0.4685 1.216  1.0865 3.0738]
done !
